In [1]:
%matplotlib notebook

from math import sin, cos, radians, pi
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

font = {'family' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)

# NOTE: All units should be in SI units (meters, kilograms, etc.)

# Initial conditions
INITIAL_ANGLE = radians(45.0)
INITIAL_VELOCITY = 50
INITIAL_HEIGHT = 0.5588

# Characteristics of the ball
BALL_DIAMETER = 0.1778 # 7 inches
BALL_MASS = 0.142 # 5 ounces
BALL_DRAG_COEFFICIENT = 0.47
BALL_SPIN = -0.1 # Angular velocity in revolutions per second

# Characteristics of the world
AIR_DENSITY = 1.225
GRAVITY = 9.80665

# Simulation settings
SIM_DT = 0.01
SIM_MAX_TIME = 10.0

In [2]:
def setup_initial_values(initial_angle, initial_velocity):
    global velocity
    velocity = np.array([
        initial_velocity * cos(initial_angle),
        initial_velocity * sin(initial_angle)
    ])
    global position
    position = np.array([
        0,
        INITIAL_HEIGHT
    ])

def calc_acceleration(use_drag=False, use_magnus=False):
    # Calculate acceleration due to gravity
    result = np.array([
        0,
        -GRAVITY
    ])
    
    if use_drag:
        # Aerodynamic drag calculation from Wikipedia: https://en.wikipedia.org/wiki/Drag_equation
        force = 0.5 * BALL_DRAG_COEFFICIENT * AIR_DENSITY * (pi * (BALL_DIAMETER / 2) ** 2) * -velocity ** 2
        result += force / BALL_MASS
    if use_magnus:
        # Magnus effect calculation from NASA: https://www.grc.nasa.gov/WWW/K-12/airplane/beach.html
        force = 4.0 / 3.0 * (4.0 * pi ** 2 * BALL_DIAMETER ** 3 * BALL_SPIN * AIR_DENSITY * -velocity)
        result += force / BALL_MASS
    
    return result

def run_simulation(initial_angle=INITIAL_ANGLE, initial_velocity=INITIAL_VELOCITY, use_drag=False, use_magnus=False):
    global position
    global velocity
    setup_initial_values(initial_angle, initial_velocity)
    
    position_arr = [position]
    velocity_arr = [velocity]

    time = 0
    while position[1] > 0 and time < SIM_MAX_TIME:
        # Calculate acceleration
        acceleration = calc_acceleration(use_drag, use_magnus)
        # Use calculated acceleration to change velocity
        velocity = velocity + acceleration * SIM_DT
        # Use calculated velocity to change position
        position = position + velocity * SIM_DT

        # Record position
        position_arr.append(position)
        velocity_arr.append(velocity)

        # Increment time by dt
        time += SIM_DT
    return (position_arr, velocity_arr)

In [ ]:
angle_res = 100
rpm_res = 100

DISTANCES = np.arange(start=2,stop=30,dtype=np.float64) * 0.3048
TARGET_HEIGHT = 2.49555
TARGET_SIZE = 0.2794

dataSets = {}
for d in DISTANCES:
    x = np.empty((angle_res, rpm_res), dtype=np.float64)
    x.fill(-30)
    dataSets[d] = x

x = np.linspace(start=26, stop=66, num=angle_res)
y = np.linspace(start=2000, stop=6000, num=rpm_res)

best_initials = {}
for d in DISTANCES:
    best_initials[d] = None

for i, angle_deg in enumerate(x):
    for j, flywheel_rpm in enumerate(y):
        (positions, velocities) = run_simulation(initial_angle=radians(angle_deg), initial_velocity=0.5*flywheel_rpm/60*0.1016*pi, use_drag=True, use_magnus=True)

        for d in DISTANCES:
            dataSet = dataSets[d]
            best_initial = best_initials[d]
            for k in range(len(positions)):
                if abs(positions[k][0] - d) < 0.025 and abs(positions[k][1] - TARGET_HEIGHT) < 0.025:
                    dataSet[j][i] = velocities[k][1]
                    if best_initial is None:
                        best_initial = (angle_deg, flywheel_rpm, velocities[k][1])
                    elif abs(velocities[k][1]) < abs(best_initial[2]):
                        best_initial = (angle_deg, flywheel_rpm, velocities[k][1])
                    break
            best_initials[d] = best_initial
            dataSets[d] = dataSet

plt.figure(figsize=(260, 116))
for i, key in enumerate(best_initials):
    best_initial = best_initials[key]
    m_plt = plt.subplot(4, 7, i + 1)
    plt.pcolormesh(x, y, dataSets[DISTANCES[i]])
    m_plt.set_title(f"{int(DISTANCES[i] / 0.3048)}ft", fontsize=78)
    best_initials_msg = None
    if best_initial is None:
        best_initials_msg = f"No good initials found"
    else:
        best_initials_msg = f"Best Initials -- Angle: {round(best_initial[0], 3)}, Flywheel RPM: {round(best_initial[1], 3)}, Y Vel: {round(best_initial[2], 3)}"
    m_plt.text(0.5, -0.075, best_initials_msg, size=26, horizontalalignment='center', transform=m_plt.transAxes)
    plt.xlabel("Angle", labelpad=13, fontsize=32)
    plt.ylabel("Flywheel RPM", labelpad=13, fontsize=32)
    cbar = plt.colorbar()
    cbar.set_label("Ball Y Velocity", rotation=270, labelpad=32, fontsize=32)

plt.show()
plt.savefig('best_initials.jpg')

In [ ]:
plt.figure(figsize=(260, 116))
for i, key in enumerate(best_initials):
    m_plt = plt.subplot(4, 7, i + 1)
    m_plt.set_title(f"{int(DISTANCES[i] / 0.3048)}ft", fontsize=78)
    plt.xlabel("Distance (M)", labelpad=13, fontsize=32)
    plt.ylabel("Height (M)", labelpad=13, fontsize=32)
    best_initial = best_initials[key]
    if best_initial is not None:
        (positions, velocities) = run_simulation(
            radians(best_initial[0]),
            0.5*best_initial[1]/60*0.1016*pi,
            True,
            True
        )
        pos_x, pos_y = zip(*positions)
        vel_x, vel_y = zip(*velocities)
        plt.plot(pos_x, pos_y)
        plt.plot([DISTANCES[i], DISTANCES[i]], [TARGET_HEIGHT - TARGET_SIZE / 2, TARGET_HEIGHT + TARGET_SIZE / 2], marker="o")
    else:
        m_plt.text(0.5, 0.5, "No Good Initials", size=48, horizontalalignment='center', transform=m_plt.transAxes)
    
plt.show()
plt.savefig("trajectories.jpg")